## LAVA JATO - AN INFOGRAPHIC OF THE CORRUPTION SCANDALS IN BRAZIL

In [3]:
from os import listdir
from os.path import isfile, join

delacoes = ['./Delações/' + f for f in listdir('./Delações/') if isfile(join('./Delações/', f))]

delacoes[:10]



['./Delações/Colaboração premiada - Alberto Youssef - Depoimento 1.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 10.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 11.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 12.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 13.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 14.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 15.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 16.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 17.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 18.pdf']

In [74]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfparser import PDFPage
from pdfminer.pdfparser import *
from io import StringIO



def convert_pdf_to_txt(path):
    retstr = StringIO()
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, retstr, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    fp = open(delacoes[0], 'rb')

    parser = PDFParser(fp)
    document = PDFDocument()
    parser.set_document(document)
    document.set_parser(parser)

    for page in document.get_pages():
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

convert_pdf_to_txt(delacoes[0])

'--------------------------------~-~ -\n\nCONFIDENCIAL \nPOLICIA FEDERAL \n\nSUPERINTEND~NCIA REGIONAL NO ESTAOO 00 PARANÁ \n\nTERMO DE COLABORAÇÃO N ° 01 \n\nTERMO DE DECLARA Ç ÕES \n\nque presta ALBERTO YOUSSEF \n\nDRCOR - Delegacia Regional de Combate ao Crime Organizado \nDELEFIN - Delegacia de Repressão a C r imes con t ra o S is tema F inance i ro e Desv io de Verbas Púb l icas \n\n16402272884 Pet 5245\n\nAo(s) 02 dia(s) do mês de outubro de 2014, nesta Superintendência Regional do \nDepartamento de Polícía Federal, em Curitiba/PR, perante EDUARDO MAUAT DA SILVA, \nDelegada de Polícia Federal, Classe Especial, matrícula nO 8190, atendendo a requisição \ndo Procurador Geral da República constante do Ofício n° 1152/Gab para se proceder á \noitiva de ALBERTO YOUSSEF, brasileiro, casado, RG 3506470-21PR, CPF 532.050.659-\n72, filho de Kalim Youssef e de Antonieta Youssef, o qual firmou acordo de colaboração \nque será levado á ratificação do Procurador Geral da República, e na presen